In [ ]:
# Data Cleaning

In [1]:
import requests
import json
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
baseURL = "https://newsapi.org/v2/everything?"
total_requests=2
verbose=True

# THIS CODE WILL NOT WORK UNLESS YOU INSERT YOUR API KEY IN THE NEXT LINE
API_KEY='7623798eeb714a3083e6e9f49af33fc3'
TOPIC='wildfire prevention'

URLpost = {'apiKey': API_KEY,
            'q': '+'+TOPIC,
            'sortBy': 'relevancy',
            'totalRequests': total_requests}

#GET DATA FROM API
response = requests.get(baseURL, URLpost) #request data from the server
# print(response.url);  
response = response.json() #extract txt data from request into json

In [3]:
def string_cleaner(input_string):
    out = ''
    try: 
        out=re.sub(r"""
                    [,.;@#?!&$-]+  # Accept one or more copies of punctuation
                    \ *           # plus zero or more copies of a space,
                    """,
                    " ",          # and replace it with a single space
                    input_string, flags=re.VERBOSE)

        #REPLACE SELECT CHARACTERS WITH NOTHING
        out = re.sub('[’.]+', '', input_string)

        #ELIMINATE DUPLICATE WHITESPACES USING WILDCARDS
        out = re.sub(r'\s+', ' ', out)

        #CONVERT TO LOWER CASE
        out=out.lower()
    except:
        print("ERROR")
        out=''
    return out

In [4]:
article_list=response['articles']   #list of dictionaries for each article
article_keys=article_list[0].keys()
print("AVAILABLE KEYS:")
print(article_keys)
index=0
cleaned_data=[];  
for article in article_list:
    tmp=[]
    if(verbose):
        print("#------------------------------------------")
        print("#",index)
        print("#------------------------------------------")

    for key in article_keys:
        if(verbose):
            print("----------------")
            print(key)
            print(article[key])
            print("----------------")

        if(key=='source'):
            src=string_cleaner(article[key]['name'])
            tmp.append(src) 

        if(key=='author'):
            author=string_cleaner(article[key])
            #ERROR CHECK (SOMETIMES AUTHOR IS SAME AS PUBLICATION)
            if(src in author): 
                print(" AUTHOR ERROR:",author);author='NA'
            tmp.append(author)

        if(key=='title'):
            tmp.append(string_cleaner(article[key]))

        if(key=='description'):
            tmp.append(string_cleaner(article[key]))

        if(key=='content'):
            tmp.append(string_cleaner(article[key]))

        if(key=='publishedAt'):
            #DEFINE DATA PATERN FOR RE TO CHECK  .* --> wildcard
            ref = re.compile('.*-.*-.*T.*:.*:.*Z')
            date=article[key]
            if(not ref.match(date)):
                print(" DATE ERROR:",date); date="NA"
            tmp.append(date)

    cleaned_data.append(tmp)
    index+=1

AVAILABLE KEYS:
dict_keys(['source', 'author', 'title', 'description', 'url', 'urlToImage', 'publishedAt', 'content'])
#------------------------------------------
# 0
#------------------------------------------
----------------
source
{'id': None, 'name': 'Yahoo Entertainment'}
----------------
----------------
author
AFP
----------------
----------------
title
Greece blaze is 'largest wildfire ever recorded in EU'
----------------
----------------
description
A forest blaze in Greece is "the largest wildfire ever recorded in the EU" and the bloc is mobilising nearly half its firefighting air wing to tackle it, a...
----------------
----------------
url
https://news.yahoo.com/greece-blaze-largest-wildfire-ever-171658134.html
----------------
----------------
urlToImage
https://s.yimg.com/ny/api/res/1.2/sjNBRNcEOvYe9pYIn4teoQ--/YXBwaWQ9aGlnaGxhbmRlcjt3PTEyMDA7aD02NzU-/https://s.yimg.com/hd/cp-video-transcode/production/f815346a-ebbe-3139-b33c-6816f3540665/2023-08-29/17-26-42/c44915c6-44

In [5]:
df = pd.DataFrame(cleaned_data)
print(df)
df.to_csv('cleaned.csv', index=False) #,index_label=['title','src','author','date','description'])

                       0                               1  \
0    yahoo entertainment                             afp   
1               bbc news  https://wwwfacebookcom/bbcnews   
2    scientific american                  curtis abraham   
3                physorg              roland lloyd parry   
4                    vox                      kylie mohr   
..                   ...                             ...   
95           screen rant                 joe deckelmeier   
96    the times of india                             afp   
97          investingcom                         reuters   
98  legalinsurrectioncom                  leslie eastman   
99            prnewswire                                   

                                                    2  \
0   greece blaze is 'largest wildfire ever recorde...   
1   voice referendum: lies fuel racism ahead of au...   
2   large herbivores can help prevent massive wild...   
3   climate change boosts risk of extreme wildfire.